In [ ]:
import sys
import io
import os
import time
import traceback
import warnings
import random
import itertools
from datetime import datetime, timedelta
from difflib import SequenceMatcher

import requests
import pandas as pd
import numpy as np
from scipy.stats import poisson

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import (
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    VotingClassifier,
    StackingClassifier
)
warnings.filterwarnings("ignore")

try:
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')
except:
    pass


In [ ]:
DEBUG_MODE = True

# Last matchday played
DEBUG_MATCHDAYS = {'SA': 15, 'PL': 16, 'PD': 16, 'BL1': 14, 'FL1': 16}


In [ ]:
def get_odds_mapping():
    return {
        'SA': [
            {'home': 'Lazio', 'away': 'Cremonese', '1': 1.60, 'X': 3.80, '2': 5.75, '1X': 1.12, '2X': 2.25, 'GG': 1.95, 'NG': 1.77},
            {'home': 'Juventus', 'away': 'Roma', '1': 2.05, 'X': 3.20, '2': 3.90, '1X': 1.24, '2X': 1.73, 'GG': 1.95, 'NG': 1.77},
            {'home': 'Cagliari', 'away': 'Pisa', '1': 2.10, 'X': 3.15, '2': 3.55, '1X': 1.27, '2X': 1.67, 'GG': 1.97, 'NG': 1.77},
            {'home': 'Sassuolo', 'away': 'Torino', '1': 2.30, 'X': 3.05, '2': 3.25, '1X': 1.30, '2X': 1.57, 'GG': 1.80, 'NG': 1.90},
            {'home': 'Fiorentina', 'away': 'Udinese', '1': 2.15, 'X': 3.20, '2': 3.40, '1X': 1.30, '2X': 1.65, 'GG': 1.80, 'NG': 1.90},
            {'home': 'Genoa', 'away': 'Atalanta', '1': 3.90, 'X': 3.35, '2': 1.92, '1X': 1.80, '2X': 1.22, 'GG': 1.85, 'NG': 1.87},
            {'home': 'Napoli', 'away': 'Parma', '1': 0, 'X': 0, '2': 0, '1X': 0, '2X': 0, 'GG': 0, 'NG': 0},
            {'home': 'Inter', 'away': 'Lecce', '1': 0, 'X': 0, '2': 0, '1X': 0, '2X': 0, 'GG': 0, 'NG': 0},
            {'home': 'Verona', 'away': 'Bologna', '1': 0, 'X': 0, '2': 0, '1X': 0, '2X': 0, 'GG': 0, 'NG': 0},
            {'home': 'Como', 'away': 'Milan', '1': 0, 'X': 0, '2': 0, '1X': 0, '2X': 0, 'GG': 0, 'NG': 0},
        ],
        'PL': [
            {'home': 'Newcastle', 'away': 'Chelsea', '1': 2.65, 'X': 3.50, '2': 2.50, '1X': 1.50, '2X': 1.43, 'GG': 1.53, 'NG': 2.35},
            {'home': 'Bournemouth', 'away': 'Burnley', '1': 1.45, 'X': 4.50, '2': 6.25, '1X': 1.09, '2X': 2.60, 'GG': 1.77, 'NG': 1.92},
            {'home': 'Brighton', 'away': 'Sunderland', '1': 1.60, 'X': 3.90, '2': 5.00, '1X': 1.14, '2X': 2.20, 'GG': 1.75, 'NG': 1.97},
            {'home': 'Wolves', 'away': 'Brentford', '1': 3.60, 'X': 3.45, '2': 2.00, '1X': 1.75, '2X': 1.25, 'GG': 1.73, 'NG': 2.00},
            {'home': 'Man City', 'away': 'West Ham', '1': 1.19, 'X': 7.25, '2': 11.00, '1X': 1.02, '2X': 4.25, 'GG': 1.82, 'NG': 1.90},
            {'home': 'Tottenham', 'away': 'Liverpool', '1': 3.35, 'X': 3.70, '2': 2.00, '1X': 1.75, '2X': 1.30, 'GG': 1.53, 'NG': 2.35},
            {'home': 'Leeds', 'away': 'Crystal Palace', '1': 2.55, 'X': 3.25, '2': 2.70, '1X': 1.40, '2X': 1.47, 'GG': 1.77, 'NG': 1.95},
            {'home': 'Everton', 'away': 'Arsenal', '1': 5.50, 'X': 3.85, '2': 1.60, '1X': 2.20, '2X': 1.12, 'GG': 2.00, 'NG': 1.73},
            {'home': 'Aston Villa', 'away': 'Man United', '1': 2.10, 'X': 3.60, '2': 3.20, '1X': 1.30, '2X': 1.67, 'GG': 1.50, 'NG': 2.40},
            {'home': 'Fulham', 'away': 'Nottingham', '1': 2.25, 'X': 3.30, '2': 3.10, '1X': 1.33, '2X': 1.60, 'GG': 1.73, 'NG': 2.00},
        ],
        'PD': [
            {'home': 'Valencia', 'away': 'Mallorca', '1': 1.97, 'X': 3.25, '2': 4.00, '1X': 1.21, '2X': 1.77, 'GG': 1.95, 'NG': 1.77},
            {'home': 'Oviedo', 'away': 'Celta', '1': 3.65, 'X': 3.40, '2': 2.00, '1X': 1.75, '2X': 1.25, 'GG': 1.90, 'NG': 1.82},
            {'home': 'Levante', 'away': 'Real Sociedad', '1': 3.20, 'X': 3.40, '2': 2.15, '1X': 1.65, '2X': 1.30, 'GG': 1.67, 'NG': 2.05},
            {'home': 'Osasuna', 'away': 'Alaves', '1': 2.25, 'X': 3.00, '2': 3.45, '1X': 1.27, '2X': 1.60, 'GG': 2.05, 'NG': 1.67},
            {'home': 'Real Madrid', 'away': 'Sevilla', '1': 1.19, 'X': 6.75, '2': 12.00, '1X': 1.01, '2X': 4.25, 'GG': 1.87, 'NG': 1.85},
            {'home': 'Girona', 'away': 'Atletico', '1': 4.60, 'X': 4.00, '2': 1.65, '1X': 2.10, '2X': 1.17, 'GG': 1.67, 'NG': 2.05},
            {'home': 'Villarreal', 'away': 'Barcelona', '1': 3.25, 'X': 4.50, '2': 1.87, '1X': 1.85, '2X': 1.30, 'GG': 1.30, 'NG': 3.30},
            {'home': 'Elche', 'away': 'Rayo', '1': 2.40, 'X': 3.10, '2': 3.00, '1X': 1.35, '2X': 1.50, 'GG': 1.93, 'NG': 1.77},
            {'home': 'Real Betis', 'away': 'Getafe', '1': 1.70, 'X': 3.45, '2': 5.25, '1X': 1.13, '2X': 2.05, 'GG': 2.15, 'NG': 1.65},
            {'home': 'Athletic Club', 'away': 'Espanyol', '1': 1.70, 'X': 3.50, '2': 5.00, '1X': 1.14, '2X': 2.05, 'GG': 2.05, 'NG': 1.70},
        ],
        'BL1': [
            {'home': 'Dortmund', 'away': 'Gladbach', '1': 1.47, 'X': 4.60, '2': 5.75, '1X': 1.11, '2X': 2.55, 'GG': 1.60, 'NG': 2.20},
            {'home': 'Augsburg', 'away': 'Werder Bremen', '1': 2.25, 'X': 3.45, '2': 3.00, '1X': 1.35, '2X': 1.60, 'GG': 1.57, 'NG': 2.25},
            {'home': 'Wolfsburg', 'away': 'Freiburg', '1': 2.40, 'X': 3.45, '2': 2.75, '1X': 1.40, '2X': 1.50, 'GG': 1.60, 'NG': 2.20},
            {'home': 'Hamburger SV', 'away': 'Frankfurt', '1': 2.60, 'X': 3.50, '2': 2.50, '1X': 1.50, '2X': 1.45, 'GG': 1.50, 'NG': 2.40},
            {'home': 'Stuttgart', 'away': 'Hoffenheim', '1': 1.95, 'X': 3.85, '2': 3.35, '1X': 1.30, '2X': 1.77, 'GG': 1.40, 'NG': 2.65},
            {'home': 'Koln', 'away': 'Union Berlin', '1': 2.35, 'X': 3.25, '2': 3.00, '1X': 1.35, '2X': 1.55, 'GG': 1.70, 'NG': 2.05},
            {'home': 'Leipzig', 'away': 'Leverkusen', '1': 2.05, 'X': 3.75, '2': 3.20, '1X': 1.30, '2X': 1.70, 'GG': 1.45, 'NG': 2.60},
            {'home': 'Mainz', 'away': 'St. Pauli', '1': 1.90, 'X': 3.35, '2': 4.00, '1X': 1.21, '2X': 1.82, 'GG': 1.88, 'NG': 1.83},
            {'home': 'Heidenheim', 'away': 'Bayern', '1': 14.00, 'X': 8.75, '2': 1.13, '1X': 5.25, '2X': 1.04, 'GG': 1.85, 'NG': 1.87},
        ],
        'FL1': [
            {'home': 'Toulouse', 'away': 'Lens', '1': 2.75, 'X': 3.30, '2': 2.40, '1X': 1.50, '2X': 1.40, 'GG': 1.63, 'NG': 2.10},
            {'home': 'Monaco', 'away': 'Lyon', '1': 1.90, 'X': 3.70, '2': 3.45, '1X': 1.27, '2X': 1.77, 'GG': 1.53, 'NG': 2.35},
            {'home': 'Nice', 'away': 'Strasbourg', '1': 2.70, 'X': 3.40, '2': 2.40, '1X': 1.50, '2X': 1.40, 'GG': 1.55, 'NG': 2.30},
            {'home': 'Lille', 'away': 'Rennes', '1': 1.83, 'X': 3.55, '2': 4.00, '1X': 1.20, '2X': 1.87, 'GG': 1.55, 'NG': 2.30},
            {'home': 'Marseille', 'away': 'Nantes', '1': 1.22, 'X': 6.25, '2': 10.00, '1X': 1.02, '2X': 3.75, 'GG': 1.95, 'NG': 1.75},
            {'home': 'Le Havre', 'away': 'Angers', '1': 2.05, 'X': 3.15, '2': 3.65, '1X': 1.23, '2X': 1.67, 'GG': 1.95, 'NG': 1.75},
            {'home': 'Brest', 'away': 'Auxerre', '1': 1.85, 'X': 3.30, '2': 4.25, '1X': 1.18, '2X': 1.85, 'GG': 1.75, 'NG': 1.95},
            {'home': 'Lorient', 'away': 'Metz', '1': 1.70, 'X': 3.70, '2': 4.40, '1X': 1.17, '2X': 2.00, 'GG': 1.70, 'NG': 2.00},
            {'home': 'PSG', 'away': 'Paris FC', '1': 1.21, 'X': 6.50, '2': 10.00, '1X': 1.02, '2X': 3.85, 'GG': 1.87, 'NG': 1.80},
        ]
    }
#

## Log